### Save the records in a human readable form

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from transformers import AutoTokenizer
import json
from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, PageBreak
)
from reportlab.lib.units import inch
from xml.sax.saxutils import escape

### define some helper functions

In [2]:
def ptext(s: str) -> str:
    # Escape &, <, > so ReportLab doesn't treat them as tags
    s = "" if s is None else str(s)
    s = escape(s)
    # Convert newlines to <br/> after escaping
    return s.replace("\n", "<br/>")

def read_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                yield json.loads(line)

def safe(x, default=""):
    return default if x is None else str(x)

def truncate(s, n=400):
    s = safe(s)
    return s if len(s) <= n else s[: n - 1] + "…"

### Define main export function
* GSM8K

In [4]:
def export_outputs_pdf(
    jsonl_path: str,
    pdf_path: str,
    max_rows: int | None = None,
    only_incorrect: bool = False,
    detail_pages_for: str = "incorrect",  # "incorrect" or "all"
):
    styles = getSampleStyleSheet()
    title_style = styles["Title"]
    normal = styles["BodyText"]
    
    mono_style = ParagraphStyle(
        "mono",
        parent=normal,
        fontName="Courier",
        fontSize=8.5,
        leading=10.5,
    )

    # Landscape letter looks good for wide tables
    doc = SimpleDocTemplate(
        pdf_path,
        pagesize=landscape(letter),
        leftMargin=0.5 * inch,
        rightMargin=0.5 * inch,
        topMargin=0.45 * inch,
        bottomMargin=0.5 * inch,
        title="Model Outputs Report",
    )

    elements = []

    # --- Load rows ---
    rows = []
    for ex in read_jsonl(jsonl_path):
        if only_incorrect and ex.get("correct", False):
            continue
        rows.append(ex)
        if max_rows is not None and len(rows) >= max_rows:
            break

    # --- Header ---
    elements.append(Paragraph("Model Outputs Report", title_style))
    elements.append(Paragraph(f"Source: {safe(jsonl_path)}", normal))
    elements.append(Paragraph(f"Number of Examples: {len(rows)}", normal))
    elements.append(Spacer(1, 10))

    # --- Detail pages ---
    elements.append(PageBreak())
    elements.append(Paragraph("Details", title_style))
    elements.append(Spacer(1, 8))

    for ex in rows:
        if detail_pages_for == "incorrect" and ex.get("correct", False):
            continue

        ch = ex.get("cheatsheet", "")
        ch = ch.strip() if ch and ch.strip() else "(none)"

        gt_full = ex.get("ground_truth_answer_full", "")
        gt_full = gt_full.strip() if gt_full else "(none)"

        sol = ex.get("solver_output", "")
        sol = sol.strip() if sol else "(none)"

        elements.append(Paragraph(
            f"Question: {ptext(ex.get('row_idx'))} • "
            f"{'Correct' if ex.get('correct') else 'Incorrect'}",
            styles["Heading2"]
        ))

        elements.append(Paragraph(
            f"<b>Hint:</b> {ptext(ex.get('hint_model_id')) if ex.get('hint_model_id') else 'None'}", normal
        ))
        elements.append(Paragraph(
            f"<b>Model:</b> {ptext(ex.get('solver_model_id'))}", normal
        ))
        elements.append(Paragraph(
            f"<b>GT:</b> {ptext(ex.get('ground_truth_answer'))} "
            f"&nbsp;&nbsp; <b>Pred:</b> {ptext(ex.get('pred_extracted'))}",
            normal,
        ))

        elements.append(Spacer(1, 6))

        elements.append(Paragraph("<b>Question</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(ex.get("question", "")), normal))

        elements.append(Spacer(1, 6))
        elements.append(Paragraph("<b>Cheatsheet</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(ch), normal))

        elements.append(Spacer(1, 6))
        elements.append(Paragraph("<b>Ground Truth (full)</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(gt_full), normal))

        elements.append(Spacer(1, 6))
        elements.append(Paragraph("<b>Solver Output</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(sol), mono_style))

        elements.append(PageBreak())


    doc.build(elements)
    print(f"Wrote: {pdf_path}")

### Write outputs in _.pdf_ format

In [5]:
export_outputs_pdf(
    jsonl_path= "../runs/gsm8k/outputs.jsonl",
    pdf_path= "../runs/gsm8k/outputs_report.pdf",
    detail_pages_for= "all"  # "incorrect" or "all"
)

Wrote: ../runs/gsm8k/outputs_report.pdf


### Define main export function
* HMMT25

In [108]:
def export_outputs_pdf(
    jsonl_path: str,
    pdf_path: str,
    max_rows: int | None = None,
    only_incorrect: bool = False,
    detail_pages_for: str = "incorrect",  # "incorrect" or "all"
):
    styles = getSampleStyleSheet()
    title_style = styles["Title"]
    normal = styles["BodyText"]
    
    mono_style = ParagraphStyle(
        "mono",
        parent=normal,
        fontName="Courier",
        fontSize=8.5,
        leading=10.5,
    )

    # Landscape letter looks good for wide tables
    doc = SimpleDocTemplate(
        pdf_path,
        pagesize=landscape(letter),
        leftMargin=0.5 * inch,
        rightMargin=0.5 * inch,
        topMargin=0.45 * inch,
        bottomMargin=0.5 * inch,
        title="Model Outputs Report",
    )

    elements = []

    # --- Load rows ---
    rows = []
    for ex in read_jsonl(jsonl_path):
        if only_incorrect and ex.get("correct", False):
            continue
        rows.append(ex)
        if max_rows is not None and len(rows) >= max_rows:
            break
    print(f"Loaded {len(rows)} examples.")
    # --- Header ---
    elements.append(Paragraph("Model Outputs Report", title_style))
    elements.append(Paragraph(f"Source: {safe(jsonl_path)}", normal))
    elements.append(Paragraph(f"Number of Examples: {len(rows)}", normal))
    elements.append(Spacer(1, 10))

    # --- Detail pages ---
    elements.append(PageBreak())
    elements.append(Paragraph("Details", title_style))
    elements.append(Spacer(1, 8))

    for i, ex in enumerate(rows):
        if detail_pages_for == "incorrect" and ex.get("correct", False):
            continue
        
        solutions_meta = np.asarray(ex.get("solutions_meta", {}))
        found_any = None
        # Iterate over Abstractions (outer list)
        for abs_idx, solution_flags in enumerate(solutions_meta):
            
            # Iterate over Solutions for that abstraction (inner list)
            for sol_idx, (eos, hit) in enumerate(solution_flags):
                
                # Find the first solution that generated an EOS token
                if eos: 
                    found_any = [abs_idx, sol_idx]
                    abs = ex.get("abstractions", "")
                    
                    if abs[0] != "":
                        abs = abs[abs_idx]
                        abs = abs.strip()
                    else:
                        abs = "(none)"
                    sol = ex.get("solutions", "")[abs_idx][sol_idx]
                    sol = sol.strip() if sol else "(none)"
                    break

            if isinstance(found_any, list):
                break

        if not found_any:
            continue

        gt_full = ex.get("ground_truth_answer", "")
        gt_full = gt_full.strip() if gt_full else "(none)"
        
        
        correct = 'Correct' if f"\\boxed{{{gt_full}}}" in sol else 'Incorrect'        
        
        elements.append(Paragraph(
            f"Question: {ptext(ex.get('row_idx'))}  • "
            f"{correct}",
            styles["Heading2"]
        ))

        elements.append(Paragraph(
            f"<b>Hint:</b> {ptext(ex.get('hint_model_id')) if ex.get('hint_model_id') else 'None'}", normal
        ))
        elements.append(Paragraph(
            f"<b>Model:</b> {ptext(ex.get('solver_model_id'))}", normal
        ))
        elements.append(Paragraph(
            f"<b>GT:</b> {ptext(ex.get('ground_truth_answer'))} ",
            normal,
        ))

        elements.append(Spacer(1, 6))

        elements.append(Paragraph("<b>Question</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(ex.get("question", "")), normal))

        elements.append(Spacer(1, 6))
        elements.append(Paragraph("<b>Cheatsheet</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(abs), normal))

        elements.append(Spacer(1, 6))
        elements.append(Paragraph("<b>Ground Truth (full)</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(gt_full), normal))

        elements.append(Spacer(1, 6))
        elements.append(Paragraph("<b>Solver Output</b>", styles["Heading3"]))
        elements.append(Paragraph(ptext(sol), mono_style))

        elements.append(PageBreak())


    doc.build(elements)
    print(f"Wrote: {pdf_path}")

In [109]:
export_outputs_pdf(
    jsonl_path = "../runs/hmmt25/records.jsonl",
    pdf_path = "../runs/hmmt25/outputs_report.pdf",
    max_rows = None,
    only_incorrect = False,
    detail_pages_for = "all"  # "incorrect" or "all"
)

Loaded 120 examples.
Wrote: ../runs/hmmt25/outputs_report.pdf
